### Installation

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/kaggle/severstal'
import sys
sys.path.append('lib')

In [ ]:
!pip uninstall -y opencv-python-headless 
!pip install -q opencv-python-headless==4.1.2.30

In [5]:
!pip install -U -q albumentations
!pip install -q segmentation_models_pytorch
!pip install -q -U pyyaml
!pip install -q catalyst

     |████████████████████████████████| 544 kB 5.1 MB/s 
     |████████████████████████████████| 120 kB 80.2 MB/s 


### Import

In [6]:
import numpy as np
import pandas as pd
import random
import os
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


from albumentations.pytorch.transforms import ToTensorV2
from albumentations import Normalize

from segmentation_models_pytorch import Unet, FPN
from commons import *

### Configuratioin

In [7]:
def provide_reproducibility():
    seed = 42
    torch.backends.cudnn.benchmark = False
    # torch.use_deterministic_algorithms(True)    
    random.seed(seed)
    np.random.seed(seed)    
    torch.manual_seed(seed)

provide_reproducibility()
np.set_printoptions(precision=4, floatmode='fixed', suppress=True)

### Loaders

In [8]:
with open('config/evaluation.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
# config
loaders = create_loaders(**config['dataset'])

### Models

In [ ]:
from torchvision.models import resnet50
model_cls = resnet50(pretrained=False)
model_cls.fc = nn.Linear(in_features=model_cls.fc.in_features, out_features=4, bias=True)
state = torch.load('checkpoints/classification/best_full.pth')
model_cls.eval()
model_cls.cuda()
model_cls.load_state_dict(state['model_state_dict'])

In [ ]:
# state = torch.load('logs/segmentation/fpn_efficientnet_b5/checkpoints/best_full.pth')
def create_seg_model(checkpoint):
    state = torch.load(checkpoint)
    model_seg = FPN('efficientnet-b5', in_channels=3, classes=1)
    model_seg.cuda()
    model_seg.eval()
    model_seg.load_state_dict(state['model_state_dict'])
    return model_seg

checkpoints = [
    'checkpoints/segmentation/channel_1/best_full.pth',
    'checkpoints/segmentation/channel_2/best_full.pth',
    'checkpoints/segmentation/channel_3/best_full.pth',
    'checkpoints/segmentation/channel_4/best_full.pth',
]

model_seg = [create_seg_model(cp) for cp in checkpoints]

### Dice calculator, with separation into negatives and positives

In [ ]:
def filter_by_min_size(outputs, min_size=2000):
    sum_per_class = outputs.sum(dim=[1, 2])
    for i, spc in enumerate(sum_per_class):
        outputs[i] = outputs[i] * (spc > min_size)
    return outputs

class DiceCalculator():
    def __init__(self):
        self.first_pass = True

    def get_masked_dice(self, nominator, denominator, mask):
        mask = mask.cuda()
        nominator = nominator# * mask
        denominator = denominator# * mask
        eps = 1e-7 * (denominator == 0)
        dice = (nominator + eps) / (denominator + eps)
        return dice * mask

    def update(self, outputs, targets):
        sum_func = partial(torch.sum, dim=[1, 2])
        nominator = 2 * sum_func(outputs * targets)
        targets_sum = sum_func(targets)
        outputs_sum = sum_func(outputs)
        denominator = outputs_sum + targets_sum

        if self.first_pass:
            self.first_pass = False
            self.count_pos = torch.zeros(nominator.shape).cuda()
            self.count_neg = torch.zeros(nominator.shape).cuda()
            self.count_all = torch.zeros(nominator.shape).cuda()
            self.dice_pos = torch.zeros(nominator.shape).cuda()
            self.dice_neg = torch.zeros(nominator.shape).cuda()
            self.dice_all = torch.zeros(nominator.shape).cuda()

        mask_pos = targets_sum != 0
        dp = self.get_masked_dice(nominator, denominator, mask_pos)
        self.dice_pos += dp
        self.count_pos += mask_pos

        mask_neg = targets_sum == 0
        dn = self.get_masked_dice(nominator, denominator, mask_neg)
        self.dice_neg += dn
        self.count_neg += mask_neg

        mask_all = torch.ones(targets_sum.shape).cuda()
        da = self.get_masked_dice(nominator, denominator, mask_all)
        self.dice_all += da
        self.count_all += mask_all

    def compute(self):
        dice_pos_mean = (self.dice_pos / self.count_pos).cpu().numpy()
        dice_neg_mean = (self.dice_neg / self.count_neg).cpu().numpy()
        dice_all_mean = (self.dice_all / self.count_all).cpu().numpy()
        return np.array([dice_pos_mean, self.count_pos.cpu().numpy(), 
                         dice_neg_mean, self.count_neg.cpu().numpy(), 
                         dice_all_mean, self.count_all.cpu().numpy()])


### Evaluation pipeline: classification with subsequent segmentation for each defect separately

In [ ]:
# using threshold estimated by f-measure maximization
thresholds = torch.tensor([0.2825, 0.1701, 0.3642, 0.3974]).cuda()

def evaluate():
    dice_calculator = DiceCalculator()
    ds = loaders['valid'].dataset
    with torch.no_grad():
        for i in tqdm(range(len(ds))):
            batch = ds[i]
            image = batch['image'].unsqueeze(0).cuda()
            mask_onehot = batch['mask_onehot'].cuda()
            outputs_cls = torch.sigmoid(model_cls.forward(image).squeeze())
            outputs_cls = (outputs_cls > thresholds)
            outputs_seg = torch.zeros(mask_onehot.shape).cuda()
            for i in range(4):
                if outputs_cls[i]:
                    outputs_seg[i] = torch.sigmoid(model_seg[i](image)) > 0.5
                # outputs_seg = filter_by_min_size(outputs_seg, 2000)
            dice_calculator.update(outputs_seg, mask_onehot)
        dice_table = dice_calculator.compute()
        print(dice_table)
        print(dice_table[4].mean())
        return dice_calculator

dc = evaluate()